## Use the OpenWeatherMap API to pull Air Quality Data

We will read in the Happiness Index resource to pull the name of every unique country and then use the [OpenWeatherMap API](https://openweathermap.org/) to find the Air Quality measurements for each country for each year, 2005-2021.

In [1]:
# Dependencies.
import pandas as pd
import numpy as np
import requests
import time, datetime
from countryinfo import CountryInfo
from config import owm_api, mq_api

In [2]:
# Read in the World Happiness Index for country names.
df = pd.read_csv('resources/whr/world-happiness-report.csv')
df.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [3]:
# Store unique country names.
countries = df['Country name'].unique().tolist()
countries[:5]

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina']

Need latitude and longitude for each country's **CAPITAL** for the API call to OpenWeatherMap, so will use the [CountryInfo](https://pypi.org/project/countryinfo/) Python module.

In [4]:
# Get list of latitude and longitude values, for country errors use NaN.
latlngs = []
for country in countries:
    try:
        latlng = CountryInfo(country).info()['capital_latlng']
        latlngs.append([country, latlng[0], latlng[1]])
    except:
        print('CountryInfo Invalid Country: ', country)
        latlngs.append([country, np.nan, np.nan])

CountryInfo Invalid Country:  Congo (Brazzaville)
CountryInfo Invalid Country:  Congo (Kinshasa)
CountryInfo Invalid Country:  Gambia
CountryInfo Invalid Country:  Hong Kong S.A.R. of China
CountryInfo Invalid Country:  Kosovo
CountryInfo Invalid Country:  Montenegro
CountryInfo Invalid Country:  Myanmar
CountryInfo Invalid Country:  North Cyprus
CountryInfo Invalid Country:  North Macedonia
CountryInfo Invalid Country:  Palestinian Territories
CountryInfo Invalid Country:  Somaliland region
CountryInfo Invalid Country:  Taiwan Province of China


In [5]:
# Add in missing ones manually.
missing_countries = [
    ['Congo (Brazzaville)', -4.4419, 15.2663],
    ['Congo (Kinshasa)', -4.322447, 15.307045],
    ['Gambia', 13.4544, -16.5753],
    ['Hong Kong S.A.R. of China', 22.302711, 114.177216],
    ['Kosovo', 42.667542, 21.166191],
    ['Montenegro', 42.4304, 19.2594],
    ['Myanmar', 19.7633, 96.0785],
    ['North Cyprus', 35.1925, 33.3598],
    ['North Macedonia', 41.9981, 21.4254],
    ['Palestinian Territories', 31.898043, 35.204269],
    ['Somaliland region', 9.5624, 44.0770],
    ['Taiwan Province of China', 25.0330, 121.5654]
]
# Loop through and replace missing values.
for x in latlngs:
    for y in missing_countries:
        if x[0] == y[0]:
            x[1] = y[1]
            x[2] = y[2]

In [6]:
# Create a DataFrame from the country, latitude, and longitude values.
countries_df = pd.DataFrame(latlngs, columns=['Country', 'Latitude', 'Longitude'])
countries_df.head()

,Country,Latitude,Longitude
0,Afghanistan,34.526011,69.177684
1,Albania,41.326873,19.818791
2,Algeria,36.775361,3.060188
3,Angola,-8.827270,13.243951
4,Argentina,-34.607568,-58.437089


Need to use UNIX time for API call, so will create list of dates to convert, 1st June to 1st July, for 2005-2021.

In [7]:
# Create list of dates for UNIX time conversion.
dates = []
for i in range(5, 22):
    if i // 10:
        dates.append([f'01/06/20{i}', f'01/07/20{i}'])
    else:
        dates.append([f'01/06/200{i}', f'01/07/200{i}'])

unix_times = []
for date in dates:
    unix_times.append(
        [int(time.mktime(datetime.datetime.strptime(date[0], "%d/%m/%Y").timetuple())),
         int(time.mktime(datetime.datetime.strptime(date[1], "%d/%m/%Y").timetuple()))]
    )
    
unix_times[:5]

[[1117598400, 1120190400],
 [1149134400, 1151726400],
 [1180670400, 1183262400],
 [1212292800, 1214884800],
 [1243828800, 1246420800]]